### <Center> Лабораторна робота №9. <br> Оцінка інформативності ознак за допомогою градієнтного бустінгу

Детальних інструкцій не буде, завдання буде описано тезисно. Рекомендується використати метод Xgboost чи Catboost, та визначити інформативність ознак.

In [ ]:
"""
#Uncomment next sections if use in google collab
!pip install google-colab
!pip install pydrive
"""

In [30]:
"""
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""

In [31]:
"""
downloaded = drive.CreateFile({'id':"1SZ0WRgGd3hygjs7yCuW6nbXpOhNCE2Wj"})
downloaded.GetContentFile('flight_delays_train.csv')

downloaded = drive.CreateFile({'id':"1n_ZXhD0v5iZHw6pAteL-NFJEc7ytIkVO"})
downloaded.GetContentFile('flight_delays_test.csv')
"""

In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [33]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [34]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [35]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Необхідно за часом вильоту літака, коду авіакомпанії-перевізника, місцю вильоту і прильоту і відстанню між аеропортами вильоту і прильоту зпрогнозувати затримку вильоту більше 15 хвилин. 

## Завдання 1. Створити вибірку :
* ознаки Distance і DepTime використати без змін;
* створити ознаку "маршрут" на  основі Origin та Dest;
* до ознак Month, DayofMonth, DayOfWeek, UniqueCarrier і "маршрут" застосувати OHE-перетворення (LabelBinarizer);

In [36]:
from sklearn.preprocessing import LabelBinarizer


def build_binary_features(data, column_name):
    binarizer = LabelBinarizer()
    values = binarizer.fit_transform(data[column_name])
    columns = (f"{column_name}_{unique_value}" for unique_value in binarizer.classes_)
    return pd.DataFrame(data=values, index=data.index, columns=columns)


In [37]:
train["Route"] = train["Origin"] + " -> " + train["Dest"]
features_to_binarize = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier"]
binarized = [build_binary_features(train, feature) for feature in features_to_binarize]
X_train = pd.concat([train[["Distance", "DepTime"]]] + binarized, axis=1)
X_train.head()

,Distance,DepTime,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,Month_c-6,Month_c-7,Month_c-8,Month_c-9,DayofMonth_c-1,DayofMonth_c-10,DayofMonth_c-11,DayofMonth_c-12,DayofMonth_c-13,DayofMonth_c-14,DayofMonth_c-15,DayofMonth_c-16,DayofMonth_c-17,DayofMonth_c-18,DayofMonth_c-19,DayofMonth_c-2,DayofMonth_c-20,DayofMonth_c-21,DayofMonth_c-22,DayofMonth_c-23,DayofMonth_c-24,DayofMonth_c-25,DayofMonth_c-26,DayofMonth_c-27,DayofMonth_c-28,DayofMonth_c-29,DayofMonth_c-3,DayofMonth_c-30,DayofMonth_c-31,DayofMonth_c-4,DayofMonth_c-5,DayofMonth_c-6,DayofMonth_c-7,DayofMonth_c-8,DayofMonth_c-9,DayOfWeek_c-1,DayOfWeek_c-2,DayOfWeek_c-3,DayOfWeek_c-4,DayOfWeek_c-5,DayOfWeek_c-6,DayOfWeek_c-7,UniqueCarrier_AA,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DH,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_HP,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_TZ,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,732,1934,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,834,1548,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,416,1422,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,872,1015,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,423,1828,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [38]:
from sklearn.feature_extraction import FeatureHasher

hasher = FeatureHasher(n_features=13, input_type="string")
values = hasher.fit_transform(train["Route"]).toarray()
route_columns = [f"route_{column_num}_feature" for column_num in range(13)]
X_train[route_columns] = values
X_train.head()


,Distance,DepTime,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,Month_c-6,Month_c-7,Month_c-8,Month_c-9,DayofMonth_c-1,DayofMonth_c-10,DayofMonth_c-11,DayofMonth_c-12,DayofMonth_c-13,DayofMonth_c-14,DayofMonth_c-15,DayofMonth_c-16,DayofMonth_c-17,DayofMonth_c-18,DayofMonth_c-19,DayofMonth_c-2,DayofMonth_c-20,DayofMonth_c-21,DayofMonth_c-22,DayofMonth_c-23,DayofMonth_c-24,DayofMonth_c-25,DayofMonth_c-26,DayofMonth_c-27,DayofMonth_c-28,DayofMonth_c-29,DayofMonth_c-3,DayofMonth_c-30,DayofMonth_c-31,DayofMonth_c-4,...,DayOfWeek_c-3,DayOfWeek_c-4,DayOfWeek_c-5,DayOfWeek_c-6,DayOfWeek_c-7,UniqueCarrier_AA,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DH,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_HP,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_TZ,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV,route_0_feature,route_1_feature,route_2_feature,route_3_feature,route_4_feature,route_5_feature,route_6_feature,route_7_feature,route_8_feature,route_9_feature,route_10_feature,route_11_feature,route_12_feature
0,732,1934,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0
1,834,1548,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,-2.0,0.0,1.0,0.0
2,416,1422,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4.0,-1.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0
3,872,1015,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5.0,0.0,-1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,423,1828,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0


In [39]:
Y_train = train["dep_delayed_15min"].apply(lambda val: 1 if val == "Y" else 0).values
Y_train

array([0, 0, 0, ..., 0, 0, 0])

## Завдання 2. Побудувати модель і оцінити інформативність ознак :
* побудувати модель на основі xgboost;
* налаштувати гіперпараметри з використанням крос-валідації;
* оцінити інформативність ознак.

In [40]:
from numpy.random import uniform
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(seed=17)
params = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
best_model = GridSearchCV(model, params, n_jobs=-1, scoring="roc_auc", cv=5)
best_model.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=17, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(3, 10, 2),
                         'min_child_weight': range(1, 6, 2)},
             pre_dispatch='2*n_jobs', refit=True, return_train_s

In [42]:
best_features = pd.DataFrame(best_model.best_estimator_.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values(by="Importance", ascending=False)
best_features.head(20)

,Importance
DepTime,0.028312
UniqueCarrier_FL,0.025938
DayofMonth_c-10,0.018138
DayofMonth_c-2,0.017629
UniqueCarrier_WN,0.017196
UniqueCarrier_EV,0.017138
UniqueCarrier_HP,0.015973
UniqueCarrier_HA,0.015457
Month_c-7,0.015447
UniqueCarrier_F9,0.014982
